In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import nltk
import os
import collections
import re
import string

import preprocessor as p
import ktrain
from ktrain import text
import nltk
from sklearn.preprocessing import LabelEncoder

C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## OFFN/HATE Predictor

In [4]:
df_full = pd.read_csv("../../preprocess_data.csv",index_col=0)
df_full = df_full.dropna()
df_full.head()

,Unnamed: 0,_id,text,task_1,task_2,text_clean
0,4986,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,HOF,PRFN,made amp amp onli abl start make money sustain...
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF,OFFN,technic still turn back clock dick head
2,1310,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,NOT,NONE,govt stop think world media liber gang ani opt...
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF,OFFN,soldier japan dick head
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF,OFFN,would better ask think sleazi shitbag lmao


In [3]:
df = df_full[df_full['task_2'].isin(['OFFN','HATE'])]
df.head()

,Unnamed: 0,_id,text,task_1,task_2,text_clean
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF,OFFN,technic still turn back clock dick head
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF,OFFN,soldier japan dick head
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF,OFFN,would better ask think sleazi shitbag lmao
8,3582,60c5d6bf5659ea5e55def54f,"@Chahal_Shekhar Sorry we won't, why can't your...",HOF,HATE,sorri wo whi rais voic thousand peopl die due ...
11,150,60c5d6bf5659ea5e55def132,"@narendramodi, you are #NotMyPrimeMinister any...",HOF,OFFN,notmyprimeminist anymor egoist populist way pl...


In [4]:
LE = LabelEncoder()
LE.fit(df['task_2'])
classes= list(LE.classes_)
classes

['HATE', 'OFFN']

In [5]:
df['task_2'] = LE.fit_transform(df['task_2'])
task1_params = dict(zip(LE.classes_, LE.transform(LE.classes_)))
df.head()

C:\Users\8888\AppData\Local\Temp\ipykernel_35004\227309049.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['task_2'] = LE.fit_transform(df['task_2'])


,Unnamed: 0,_id,text,task_1,task_2,text_clean
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF,1,technic still turn back clock dick head
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF,1,soldier japan dick head
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF,1,would better ask think sleazi shitbag lmao
8,3582,60c5d6bf5659ea5e55def54f,"@Chahal_Shekhar Sorry we won't, why can't your...",HOF,0,sorri wo whi rais voic thousand peopl die due ...
11,150,60c5d6bf5659ea5e55def132,"@narendramodi, you are #NotMyPrimeMinister any...",HOF,1,notmyprimeminist anymor egoist populist way pl...


In [6]:
LE.fit(df['task_2'])
classes= list(LE.classes_)
classes

[0, 1]

In [7]:
# split train dataset into train, validation and test sets
train_text, temp_text, train_labels, temp_labels = train_test_split(df['text_clean'], df['task_2'], 
                                                                    random_state=42, 
                                                                    test_size=0.1)

In [8]:
X_train = train_text.tolist()
X_test = temp_text.tolist()

y_train = train_labels.tolist()
y_test = temp_labels.tolist()

In [9]:
len(X_test),len(X_train)

(131, 1174)

In [10]:
import ktrain
from ktrain import text
MODEL_NAME = 'models--distilbert--distilbert-base-uncased'  # replace this with model of choice
t = text.Transformer(MODEL_NAME, maxlen=500, class_names=classes)
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)
history = learner.fit_onecycle(5e-5,2)
learner.validate(class_names=classes) # class_names must be string values

preprocessing train...
language: en
train sequence lengths:
	mean : 17
	95percentile : 28
	99percentile : 31


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 16
	95percentile : 27
	99percentile : 29




begin training using onecycle policy with max lr of 5e-05...
Epoch 1/2
196/196 [==============================] - 886s 4s/step - loss: 0.6504 - accuracy: 0.6056 - val_loss: 0.5972 - val_accuracy: 0.6489
Epoch 2/2
5/5 [==============================] - 4s 640ms/step
              precision    recall  f1-score   support

           0       0.70      0.94      0.80        68
           1       0.90      0.56      0.69        63

    accuracy                           0.76       131
   macro avg       0.80      0.75      0.74       131
weighted avg       0.79      0.76      0.75       131



array([[64,  4],
       [28, 35]], dtype=int64)

In [11]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [12]:
predictor.save('bert_OFFN_HATE')

## Data for validating Proposed Model

In [5]:
LE = LabelEncoder()
LE.fit(df_full['task_2'])
classes= list(LE.classes_)
classes

['HATE', 'NONE', 'OFFN', 'PRFN']

In [6]:
df_full['task_2'] = LE.fit_transform(df_full['task_2'])
task1_params = dict(zip(LE.classes_, LE.transform(LE.classes_)))

In [7]:
LE.fit(df_full['task_2'])
classes= list(LE.classes_)
classes

[0, 1, 2, 3]

In [8]:
# split train dataset into train, validation and test sets
train_text, temp_text, train_labels, temp_labels = train_test_split(df_full['text_clean'], df_full['task_2'], 
                                                                    random_state=42, 
                                                                    test_size=0.3)

In [9]:
X_train = train_text.tolist()
X_test = temp_text.tolist()

y_train = train_labels.tolist()
y_test = temp_labels.tolist()

## Step 1: HOF/NOT Prediction

In [11]:
reloaded_predictor_A = ktrain.load_predictor('bert_A')

In [12]:
len(X_test),len(y_test)

(1152, 1152)

In [13]:
label = []
i = 0
for tweet in X_test:
    ans = reloaded_predictor_A.predict(tweet)
    i += 1
    #print(i,len(tweet))
    if (ans == 0):
        label.append('HOF')
    else:
        label.append('NOT')

In [14]:
for i in range(10):
    print(label[i],X_test[i])

HOF woman get pretti high bodi count onli take one dick time virgin indic wisdom good head one shoulder doe correl invers woman bodi count
NOT lost one belov teacher school covid19 last week littl tribut teacher influenc life rip sir miss teacherappreciationweek covidindia indiacovidcrisi
NOT usa first ladi wife command chief uk first ladi command chief elizabeth ii carri antoinett mere latest shag probabl queen prime minist
HOF look bastard toad dog conchetumar son donkey jerk go sing precis concis keep fuck go make levit leg slit cut webeo
HOF bloodi high definit multimedia interfac bloodi well fuck understand got hell bitch
HOF govern ca manag defeat vaccin oxygen foreign aid pandem right remain power resign pm modi save loss live
NOT want covid19 giver help us ani way thank chinesevirus ccpvirus
HOF alright fucker pqrted
HOF grown dont know mani time announc fuck wan na fuck love wan na love wan na shut fuck talk
HOF hmmmm bitch


In [15]:
y_test[:10]

[3, 1, 1, 3, 3, 0, 1, 3, 1, 3]

## Step 2: Profanity Check

In [17]:
from profanityfilter import ProfanityFilter
pf = ProfanityFilter()

In [19]:
from better_profanity import profanity

In [20]:
len(label),len(X_test),len(y_test)

(1152, 1152, 1152)

In [21]:
op = []
for i in range(len(X_test)):
    # Not even HOT
    if(label[i] == 'NOT'):
        op.append('NONE')
    else:
        if(profanity.contains_profanity(X_test[i]) == True):
            op.append('PRFN')
        else:
            if(pf.is_profane(X_test[i]) == True):
                op.append('PRFN')
            # Not PRNF but can be OFFN/HATE
            else:
                op.append('HOF')

In [22]:
len(label),len(op),len(X_test),len(y_test)

(1152, 1152, 1152, 1152)

## Step 3: Predict OFFN/HATE

In [23]:
reloaded_predictor_OH = ktrain.load_predictor('bert_OFFN_HATE')

In [24]:
for i in range(len(op)):
    if (op[i] == 'HOF'):
        ans = reloaded_predictor_OH.predict(X_test[i])
        if (ans == 0):
            op[i] = 'HATE'
        else:
            op[i] = 'OFFN'


In [25]:
LE.fit(op)
list(LE.classes_)

['HATE', 'NONE', 'OFFN', 'PRFN']

In [26]:
op = LE.fit_transform(op)
LE.fit(op)
list(LE.classes_)

[0, 1, 2, 3]

## Check for accuracy

In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, op)

0.6579861111111112

In [28]:
from sklearn import metrics
print(metrics.classification_report(y_test,op))

              precision    recall  f1-score   support

           0       0.58      0.54      0.56       209
           1       0.73      0.69      0.71       382
           2       0.61      0.33      0.43       197
           3       0.64      0.87      0.74       364

    accuracy                           0.66      1152
   macro avg       0.64      0.61      0.61      1152
weighted avg       0.66      0.66      0.64      1152



## Running the model on test data

In [29]:
df_ans = pd.read_csv("../../preprocess_test_data.csv",index_col=0)
df_ans['text_clean'] = df_ans['text_clean'].fillna('neutral')
df_ans.head()

,_id,text,text_clean
0,60c5d6bf5659ea5e55deffcb,Fewer people coming in for vaccinations. So sa...,fewer peopl come vaccin sad nurs covidvaccin vumc
1,60c5d6bf5659ea5e55df028c,@MattHancock This may all be true. But... What...,may true piss big dom
2,60c5d6bf5659ea5e55def377,@Layla_EFC I’ve unfollowed him the wanker,unfollow wanker
3,60c5d6bf5659ea5e55def4c7,You guys are losing it all over the world. The...,guy lose world jung label islamophobia
4,60c5d6bf5659ea5e55df01a6,"And thus death laughs... It is sad merriment, ...",thus death laugh sad merriment still covid ind...


In [30]:
label = []
i = 0
for tweet in df_ans['text_clean']:
    ans = reloaded_predictor_A.predict(tweet)
    i += 1
    #print(i,len(tweet))
    if (ans == 0):
        label.append('HOF')
    else:
        label.append('NOT')

In [31]:
len(label),len(df_ans)

(1281, 1281)

In [32]:
op = []
for i in range(len(df_ans['text_clean'])):
    # Not even HOT
    if(label[i] == 'NOT'):
        op.append('NONE')
    else:
        if(profanity.contains_profanity(df_ans['text_clean'][i]) == True):
            op.append('PRFN')
        else:
            if(pf.is_profane(df_ans['text_clean'][i]) == True):
                op.append('PRFN')
            # Not PRNF but can be OFFN/HATE
            else:
                op.append('HOF')

In [33]:
len(label),len(df_ans),len(op)

(1281, 1281, 1281)

In [34]:
for i in range(len(op)):
    if (op[i] == 'HOF'):
        ans = reloaded_predictor_OH.predict(df_ans['text_clean'][i])
        if (ans == 0):
            op[i] = 'HATE'
        else:
            op[i] = 'OFFN'

In [35]:
LE = LabelEncoder()
LE.fit(op)
list(LE.classes_)

['HATE', 'NONE', 'OFFN', 'PRFN']

In [36]:
df_ans = df_ans.drop(['text','text_clean'],axis=1)
df_ans = df_ans.reset_index()
df_ans = df_ans.drop(['index'],axis=1)
df_ans['label'] = pd.Series(op)

In [37]:
df_ans.head()

,_id,label
0,60c5d6bf5659ea5e55deffcb,NONE
1,60c5d6bf5659ea5e55df028c,PRFN
2,60c5d6bf5659ea5e55def377,PRFN
3,60c5d6bf5659ea5e55def4c7,NONE
4,60c5d6bf5659ea5e55df01a6,NONE


In [38]:
df_ans.to_csv('Proposed_BERT_Model.csv',index=False)